In [6]:
#!pip install tf-nightly

In [7]:
#!pip install tensorflow_datasets

In [115]:
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
# encoding=utf8
from importlib import reload
import sys
reload(sys)
#%tensorflow_version 2.x
#load tensorboard
#%load_ext tensorboard
 #%tensorboard --logdir logs
%autosave 5

Autosaving every 5 seconds


In [2]:
import os
print(tf.__version__)
print(os.getcwd())



2.3.0-dev20200516
/home/nanaboateng/notebooks/NLP


In [116]:
with open('/home/nanaboateng/notebooks/Data/jw300.en-tw.tw', 'r') as f:
     #Twi_data = f.readlines()
      Twi_data= f.read().splitlines()

In [117]:
english_data = [line.rstrip() for line in open('/home/nanaboateng/notebooks/Data/jw300.en-tw.en')]
english_data[0:5]

['“ Oh , Jehovah , Keep My Young Girl Faithful ! ”',
 'I WAS born in 1930 in Alsace , France , into an artistic family .',
 'During the evenings , Father , sitting in his lounge chair , would be reading some books about geography or astronomy .',
 'My doggy would be sleeping by his feet , and Daddy would be sharing with Mum some highlights from his reading while she was knitting for her family .',
 'How much I enjoyed those evenings !']

In [118]:
def normalize_eng(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

def normalize_twi(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.ƆɔɛƐ!?’]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s


# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence_english(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  #w = re.sub(r'[^Ɔ-ɔɛƐ]+', r' ', w)
  #strip() Parameters
  #chars (optional) - a string specifying the set of characters to be removed.
  #If the chars argument is not provided, all leading and trailing whitespaces are removed from the string.
  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  #w = '<start> ' + w + ' <end>'
  return w




  
def preprocess_sentence_twi(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-ZɛƐɔƆ?.!,¿]+", " ", w)
  #w = re.sub(r'[^Ɔ-ɔɛƐ]+', r' ', w)
  #strip() Parameters
  #chars (optional) - a string specifying the set of characters to be removed.
  #If the chars argument is not provided, all leading and trailing whitespaces are removed from the string.
  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  #w = '<start> ' + w + ' <end>'
  return w

In [119]:
english_d  = list(map(preprocess_sentence_english,english_data))
english_d[0:5]

['oh , jehovah , keep my young girl faithful !',
 'i was born in in alsace , france , into an artistic family .',
 'during the evenings , father , sitting in his lounge chair , would be reading some books about geography or astronomy .',
 'my doggy would be sleeping by his feet , and daddy would be sharing with mum some highlights from his reading while she was knitting for her family .',
 'how much i enjoyed those evenings !']

In [120]:
twi_d  = list(map(preprocess_sentence_twi,Twi_data))
twi_d[0:5]

['oo , yehowa , boa me babea kumaa yi ma onni nokware !',
 'wɔwoo me too abusua a wonim adwinne di mu wɔ alsace , france , wɔ mu .',
 'na papa taa pa twere n agua mu kenkan asase ho nsɛm anaa ewim nneɛma ho nhoma bi anwummere anwummere .',
 'na me kraman da papa nan so , na na ɔka nsɛntitiriw a epue wɔ n akenkan no mu kyerɛ maame bere a ɔnwene abusua no nneɛma no .',
 'na m ani gye anwummere a ɛtete saa no ho kɛse !']

In [123]:
#all_d= [english_d[0:5],twi_d[0:5]]
all_d=list(zip(english_d,twi_d))
#all_d = {'en':english_d,'tw':twi_d}
all_d[0:5]


[('oh , jehovah , keep my young girl faithful !',
  'oo , yehowa , boa me babea kumaa yi ma onni nokware !'),
 ('i was born in in alsace , france , into an artistic family .',
  'wɔwoo me too abusua a wonim adwinne di mu wɔ alsace , france , wɔ mu .'),
 ('during the evenings , father , sitting in his lounge chair , would be reading some books about geography or astronomy .',
  'na papa taa pa twere n agua mu kenkan asase ho nsɛm anaa ewim nneɛma ho nhoma bi anwummere anwummere .'),
 ('my doggy would be sleeping by his feet , and daddy would be sharing with mum some highlights from his reading while she was knitting for her family .',
  'na me kraman da papa nan so , na na ɔka nsɛntitiriw a epue wɔ n akenkan no mu kyerɛ maame bere a ɔnwene abusua no nneɛma no .'),
 ('how much i enjoyed those evenings !',
  'na m ani gye anwummere a ɛtete saa no ho kɛse !')]

In [124]:
# Creating training and validation sets using an 80-20 split
train_examples, val_examples = train_test_split(all_d, test_size=0.2)


In [ ]:
def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.string)
  return arg


In [107]:
#train_examples = np.array(train_examples)
#val_examples = np.array(val_examples)

train_examples = tf.constant(train_examples)
#val_examples = tf.constant(val_examples)
#train_examples = my_func(train_examples)
#val_examples = my_func(val_examples)
#train_examples = tfds.as_numpy(train_examples)
#val_examples =   tfds.as_numpy(val_examples)
# tf.convert_to_tensor
train_examples = tf.data.Dataset.from_tensor_slices(train_examples)
#val_examples = tf.data.Dataset.from_tensor_slices(val_examples)
train_examples

<TensorSliceDataset shapes: (2,), types: tf.string>

In [126]:
#print(train_examples.shape)
#train_examples[0:5,:]
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en  for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt  for pt, en in train_examples), target_vocab_size=2**13)

In [128]:
print(tokenizer_pt.vocab_size)
print(tokenizer_en.vocab_size)

8266
8149


In [125]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

AttributeError: 'str' object has no attribute 'numpy'

save tokens

In [15]:
#save
filename = '/home/nanaboateng/notebooks/Data/tokenizer'
tokenizer_en.save_to_file(filename)

# Load
tokenizer_en = tfds.features.text.SubwordTextEncoder.load_from_file(filename)

In [87]:
sample_string = 'Transformer is awesome.'

tokenized_string = tokenizer_en.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer_en.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [5360, 5390, 5373, 5386, 5391, 5378, 5387, 5390, 289, 5390, 5308, 4432, 4882, 40, 289, 5322]
The original string: Transformer is awesome.


The tokenizer encodes the string by breaking it into subwords if the word is not in its dictionary.

In [88]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_en.decode([ts])))

5360 ----> T
5390 ----> r
5373 ----> a
5386 ----> n
5391 ----> s
5378 ----> f
5387 ----> o
5390 ----> r
289 ----> me
5390 ----> r
5308 ---->  
4432 ----> is 
4882 ----> awe
40 ----> so
289 ----> me
5322 ----> .


In [89]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64

Add a start and end token to the input and target.

In [108]:
def encode(lang1, lang2):
  lang1 = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(
      lang1.numpy()) + [tokenizer_pt.vocab_size+1]

  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1]
  
  return lang1, lang2

You want to use Dataset.map to apply this function to each element of the dataset. Dataset.map runs in graph mode.

- Graph tensors do not have a value.
- In graph mode you can only use TensorFlow Ops and functions.
So you can't .map this function directly: You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function

In [109]:
def tf_encode(pt, en):
  result_pt, result_en = tf.py_function(encode, [pt, en], [tf.int64, tf.int64])
  result_pt.set_shape([None])
  result_en.set_shape([None])

  return result_pt, result_en

In [113]:
def tf_encode(pt):
  result_pt = tf.py_function(encode, [pt], [tf.int64])
  result_pt.set_shape([None])

  return result_pt

Note: To keep this example small and relatively fast, drop examples with a length of over 40 tokens.

In [110]:
MAX_LENGTH = 40

In [111]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [114]:
train_preprocessed = (
    train_examples
    .map(tf_encode) 
    .filter(filter_max_length)
    # cache the dataset to memory to get a speedup while reading from it.
    .cache()
    .shuffle(BUFFER_SIZE))

val_preprocessed = (
    val_examples
    .map(tf_encode)
    .filter(filter_max_length))        

AttributeError: in user code:

    <ipython-input-113-bc9f9b7556a0>:3 tf_encode  *
        result_pt.set_shape([None])

    AttributeError: 'list' object has no attribute 'set_shape'


In [28]:
import tensorflow as tf

my_str = tf.constant(train_examples[0:5,:])
my_str_npy = my_str.numpy()

print(my_str_npy)
type(my_str_npy)

[[b'for our four teachers , i had to cook meat , bake cakes , and prepare vegetables .'
  b'na \xc9\x9bs\xc9\x9b s\xc9\x9b menoa nam , to \xc9\x94fam , na misiesie nhabamma ma y\xc9\x9bn akyer\xc9\x9bkyer\xc9\x9bfo baanan no .']
 [b'her findings , in the book traits of a healthy family , discuss the top qualities selected by the experts .'
  b'\xc9\x94kaa nea ohui no ho as\xc9\x9bm w\xc9\x94 nhoma traits of a healthy family mu , na na \xc9\x9bfa su atitiriw a animdefo no pawee no ho .']
 [b'proverbs says by wisdom a household will be built up , and by discernment it will prove firmly established .'
  b'mmebus\xc9\x9bm ka s\xc9\x9b w\xc9\x94de nyansa na esi dan , na w\xc9\x94de nhumu ma egyina .']
 [b'when pam has a problem , she may feel overwhelmed .'
  b's\xc9\x9b adwoa hyia \xc9\x94haw bi a , ebia \xc9\x9bb\xc9\x9bhy\xc9\x9b ne so .']
 [b'one year i gave a speech to my class about my religion .'
  b'afe bi , memaa me som ho \xc9\x94kasa kyer\xc9\x9b\xc9\x9b me mf\xc9\x9bfo sukuufo n

numpy.ndarray

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


ValueError: Tensor conversion requested dtype float32 for Tensor with dtype string: <tf.Tensor: shape=(2, 2), dtype=string, numpy=
array([[b'hello', b'yes'],
       [b'no', b'you']], dtype=object)>

In [38]:
train_examples.dtype

tf.string

In [79]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train_examples, tf.string)
            
        )
    )
)

In [80]:
training_dataset

<TensorSliceDataset shapes: (2,), types: tf.string>

In [94]:
import tensorflow as tf
input = tf.constant([10,20,30])
ds = tf.data.Dataset.from_tensor_slices(input)
#Out: <DatasetV1Adapter shapes: (), types: tf.int32>
print(ds)
ds1=ds.map(lambda x: [x+1, x+2, x+3])
print(ds1)
#Out: <DatasetV1Adapter shapes: ((), (), ()), types: (tf.int32, tf.int32, tf.int32)>
ds2=ds.map(lambda x: [[x+1, x+2, x+3]])
print(ds2)
#Out: <DatasetV1Adapter shapes: ((3,),), types: (tf.int32,)>
ds3=ds.map(lambda x: ([x+1, x+2, x+3]))
print(ds3)
#Out: <DatasetV1Adapter shapes: ((), (), ()), types: (tf.int32, tf.int32, tf.int32)>

<TensorSliceDataset shapes: (), types: tf.int32>
<MapDataset shapes: ((), (), ()), types: (tf.int32, tf.int32, tf.int32)>
<MapDataset shapes: ((3,),), types: (tf.int32,)>
<MapDataset shapes: ((), (), ()), types: (tf.int32, tf.int32, tf.int32)>


In [101]:
import tensorflow as tf

def real_gen():
    for _ in range(10):
        yield 10.0

dataset = tf.data.Dataset.from_generator(real_gen, (tf.float32))
# Fails
#print(next(dataset))
real_gen
dataset

<FlatMapDataset shapes: <unknown>, types: tf.float32>